<a href="https://colab.research.google.com/github/samsonharveyy/object-detection/blob/master/video_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch -U
!pip install torchvision -U

In [3]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.11.0+cu113 False


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import gdown
import tarfile

#download pretrained weights 
#function also found as download_trained() in download.py in repo
def download_trained():
    #single file, no extraction
    #(for 60 epochs only)
    #gdown.download("https://drive.google.com/uc?id=18q5n_IUcjeOrMf6q2LIX_MvD2ee6bsZm", "drinks-trained-weights.pth", quiet=False)

    #updated version, 80 epochs
    gdown.download("https://drive.google.com/uc?id=1yF7MCj116-hc16xshxBjaJgU9zT_by8C", "drinks-trained-weights.pth", quiet=False)

def download_

download_trained()

Downloading...
From: https://drive.google.com/uc?id=1yF7MCj116-hc16xshxBjaJgU9zT_by8C
To: /content/drinks-trained-weights.pth
100%|██████████| 76.1M/76.1M [00:00<00:00, 160MB/s]


In [11]:
#py files from my repo to help with object detection
import sys 
sys.path.insert(0,"/content/drive/My Drive/detection_files")

In [12]:
#from download import download_data
#from download import download_trained
import os

#need to upload model_helper.py from repo to Colab
from model_helper import segment
from model_helper import ImageDataset

#need to upload other .py files from repo
#i added it to a folder in my drive called detection_files
import engine
import utils
import label_utils

import torch
import torchvision
from torchvision import transforms
from PIL import Image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def main():

    if not os.path.exists("/content/drinks-trained-weights.pth"):
        download_trained()

    #for Google Colab (need to change path)
    labels_test_path = "/content/drive/My Drive/drinks/labels_test.csv"
    labels_train_path = "/content/drive/My Drive/drinks/labels_train.csv"
    #labels_test_path = "drinks/labels_test.csv"
    #labels_train_path = "drinks/labels_train.csv"
    test_dict, test_classes = label_utils.build_label_dictionary(labels_test_path)
    train_dict, train_classes = label_utils.build_label_dictionary(labels_train_path)
    
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    num_classes = 4
    test_split = ImageDataset(test_dict, transforms.ToTensor())

    test_loader = torch.utils.data.DataLoader(
        test_split, 
        batch_size = 1, 
        shuffle = False, 
        num_workers = 2,
        collate_fn = utils.collate_fn)

    model = segment(num_classes)

    if torch.cuda.is_available():
      #for google colab
      model.load_state_dict(torch.load("/content/drinks-trained-weights.pth"))
      #model.load_state_dict(torch.load("drinks-trained-weights.pth"))
    else:
      #for google colab
      model.load_state_dict(torch.load("/content/drinks-trained-weights.pth", map_location=torch.device("cpu")))
      #model.load_state_dict(torch.load("drinks-trained-weights.pth", map_location=torch.device("cpu")))

    #use either gpu or cpu
    model.to(device)

    #test model performance evaluation
    engine.evaluate(model, test_loader, device = device)
    
if __name__ == "__main__":
    main()

Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth


  0%|          | 0.00/74.2M [00:00<?, ?B/s]

creating index...
index created!
Test:  [ 0/51]  eta: 0:00:24  model_time: 0.3754 (0.3754)  evaluator_time: 0.0030 (0.0030)  time: 0.4796  data: 0.1008
Test:  [50/51]  eta: 0:00:00  model_time: 0.1431 (0.1529)  evaluator_time: 0.0016 (0.0017)  time: 0.1495  data: 0.0041
Test: Total time: 0:00:08 (0.1623 s / it)
Averaged stats: model_time: 0.1431 (0.1529)  evaluator_time: 0.0016 (0.0017)
Accumulating evaluation results...
DONE (t=0.03s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.817
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.982
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.924
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.769
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.821
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | max

In [13]:
!git clone https://github.com/opencv/opencv.git
!mkdir Video

Cloning into 'opencv'...
remote: Enumerating objects: 305219, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 305219 (delta 0), reused 0 (delta 0), pack-reused 305207
Receiving objects: 100% (305219/305219), 494.40 MiB | 27.83 MiB/s, done.
Resolving deltas: 100% (212416/212416), done.
Checking out files: 100% (7045/7045), done.


In [14]:
!pip install ffmpeg-python


from IPython.display import HTML, Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg

video_file_test = '/content/Video/test.mp4' 
  

VIDEO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var my_btn_txt = document.createTextNode("Press to start recording");

my_btn.appendChild(my_btn_txt);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, videoStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  videoStream = stream;
  var options = {  
    mimeType : 'video/webm;codecs=vp9'  
  };            
  recorder = new MediaRecorder(stream, options);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('video');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({video: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      videoStream.getVideoTracks()[0].stop();
      recordButton.innerText = "Saving the recording... Please wait!"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def start_webcam():
  js = Javascript('''
    async function startWebcam() {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      return;

    }
    ''')
  
  display(js)
  data = eval_js('startWebcam()')
  
    
start_webcam()

def get_video():
  display(HTML(VIDEO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  return binary

<IPython.core.display.Javascript object>

In [15]:
videofile = get_video()

with open(video_file_test, 'wb') as f:
  f.write(videofile)

In [ ]:
from model_helper import segment
from model_helper import ImageDataset
import torch
import imutils
import numpy
import torchvision
import glob


#pretrained model
pretrained_model = "/content/drinks-trained-weights.pth"

num_classes = 4
model = segment(num_classes)
model.load_state_dict(torch.load(pretrained_model))
model.eval()

#labels for objects and colors for bounding boxes
labels = {0: 'background', 1: 'Water', 2: 'Soda', 3: 'Pineapple Juice'}
colors = {0: (0,0,0), 1: (255,102,102), 2: (51,51,255), 3: (51,255,51)}

cap = cv2.VideoCapture(video_file_test) 

#frame array to capture frames to convert back to video later
frame_array = []

while cap.isOpened():

    ret, frame = cap.read() 

    #if no frames left, break the loop
    if not ret: 
      break

    #convert image to tensor
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.float32) / 255
    image = torchvision.transforms.ToTensor()(image)
    predictions = model([image])[0]

    #loop to add bounding box and text for detection
    for box, label, score in zip(predictions['boxes'], predictions['labels'], predictions['scores']):
        label = label.item()
        if label == 0 or score < 0.85:
            continue
        x_min, y_min, x_max, y_max = box.int().tolist()
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), colors[label], 2)
        cv2.putText(frame, f"{labels[label]} {score.item():.2f}", (x_min, y_min-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[label], 2)


    # show modified frame
    #cv2_imshow(frame)

    #verify height and width
    #print(frame.shape)

    frame_array.append(frame)


#convert back to video
#verified above that width by height is 640x480
result = "/content/Video/video_demo.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 30
frame_width = 640
frame_height = 480
output = cv2.VideoWriter(result,fourcc,fps,(frame_width,frame_height))
for i in range(len(frame_array)):
    output.write(frame_array[i])
output.release()